In [1]:
import sys
cpath = !pwd
#sys.path.append(cpath[0][:-18])
sys.path.append('C:\\Users\\iamollas\\Downloads\\LionForests Journal\\')

In [2]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler,StandardScaler, LabelEncoder
from sklearn import preprocessing
from lionforests import LionForests

In [3]:
from sklearn.datasets import load_boston
import matplotlib.pyplot as plt
data = load_boston()
X = data['data']
y = data['target']
feature_names = data['feature_names']
class_names=["House Price"]

In [4]:
X = data['data']
y = data['target']
feature_names = data['feature_names']
feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [8]:
len(X[0])

13

In [5]:
LFr = LionForests(None, False, None, list(feature_names), class_names=["House Price"], categorical_features=None)
parameters = [{
    'max_depth': [10],
    'max_features': [0.75],
    'bootstrap': [True],
    'min_samples_leaf' : [1],
    'n_estimators': [1000]
}]
LFr.fit(X, y, params=parameters)

In [6]:
LFr.error

-2.951595344069489

In [8]:
LFr.explain(X[10], reduction=True, ar_algorithm=None, cl_algorithm=None, save_plots=False, instance_qe=0, clusters=0, method = 'R1')

['if 6.178<=RM<=6.544 & 20.195<=LSTAT<=21.27 & 6.333<=DIS<=12.126 & 0.224<=CRIM<=0.524 & 7.225<=INDUS<=9.365 & 0.469<=NOX<=0.531 & 12.6<=PTRATIO<=16.55 & 301.5<=TAX<=434.5 & 93.6<=AGE<=100.0 & 390.82<=B<=392.995 then House Price: 18.3152 +/- 2.9814 error',
 100,
 13,
 91,
 10,
 [{'NOX': [['<=', 0.5730452835559845]],
   'RM': [['<=', 0.6287602782249451]],
   'PTRATIO': [['<=', 0.6648935973644257]],
   'LSTAT': [['>', 0.401903972029686]]},
  {'CRIM': [['<=', 0.07727716490626335]],
   'INDUS': [['>', 0.2479838728904724]],
   'DIS': [['>', 0.02205621637403965]],
   'LSTAT': [['>', 0.3961092680692673]]},
  {'RM': [['<=', 0.5952289700508118]],
   'CRIM': [['<=', 0.07727716490626335], ['>', 0.0023587195901200175]],
   'LSTAT': [['>', 0.47392383217811584]]},
  {'RM': [['<=', 0.6254071593284607]],
   'CRIM': [['<=', 0.08465376123785973]],
   'NOX': [['<=', 0.30041152238845825]],
   'LSTAT': [['<=', 0.7729028463363647], ['>', 0.3496136963367462]]},
  {'RM': [['<=', 0.648400068283081]],
   'NOX':

In [11]:
from tqdm import tqdm
import time

results = []
iterr = 100
with tqdm(total=5*4*2*4*5) as pbar:
    for err in [0, 0.01, 0.1, 1, 10]:
        for i in [1, 5, 7, 10]:
            for j in [500, 1000]:#500, 1000
                for o in ['sqrt', 'log2', 0.75, None]:
                    parameters = [{
                        'max_depth': [i],
                        'max_features': [o],
                        'bootstrap': [True],
                        'min_samples_leaf': [1],
                        'n_estimators': [j]
                    }]
                    lf = LionForests(None, False, None, list(feature_names), class_names=["House Price"], categorical_features=None)
                    lf.fit(X, y, params=parameters)
                    lf.error
                    tlf = 0
                    fr = 0
                    pr = 0
                    for inde in range(iterr):
                        t_a = time.time()
                        a, b, c, d, e, f, g = lf.explain(
                            X[inde], False, None, None, False)
                        fr = fr + c - e
                        pr = pr + b - d
                        tlf = tlf + time.time() - t_a
                    results.append([str(i), str(j), str(o), 'NoRed','_', '_', tlf/iterr, fr/iterr, pr/iterr, 0, 0])
                    pbar.update(1)
                    error = err*(y.max()-y.min())/100
                    if error == 0:
                        error = abs(lf.error)
                    for k in ['R1', 'R2', 'R3']:
                        if 'R1' == k:
                            for ara in ['apriori', 'fpgrowth']:
                                tlf = 0
                                fr = 0
                                pr = 0
                                lr = 0
                                for inde in range(iterr):
                                    t_a = time.time()
                                    a, b, c, d, e, f, g = lf.explain(
                                        X[inde], True, ara,  instance_qe=error, method=k)
                                    fr = fr + c - e
                                    pr = pr + b - d
                                    if g is not None:
                                        lr = lr + abs(g)
                                    tlf = tlf + time.time() - t_a
                                results.append([str(i), str(j), str(
                                    o), k, ara, '_', tlf/iterr, fr/iterr, pr/iterr, error, lr/iterr])
                                pbar.update(1)
                        else:
                            tlf = 0
                            fr = 0
                            pr = 0
                            lr = 0
                            for inde in range(iterr):
                                t_a = time.time()
                                a, b, c, d, e, f, g = lf.explain(
                                    X[inde], True, None, None,  instance_qe=error, method=k)
                                fr = fr + c - e
                                pr = pr + b - d
                                if g is not None:
                                    lr = lr + abs(g)
                                tlf = tlf + time.time() - t_a
                            results.append([str(i), str(j), str(
                                o), k, '_', '_', tlf/iterr, fr/iterr, pr/iterr, error, lr/iterr])
                            pbar.update(1)

 38%|████████████████████████████▉                                               | 305/800 [3:54:35<8:47:19, 63.92s/it]C:\Users\iamollas\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
 74%|████████████████████████████████████████████████████████                    | 590/800 [7:29:37<3:53:18, 66.66s/it]C:\Users\iamollas\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
 89%|███████████████████████████████████████████████████████████████████▉        | 715/800 [9:05:50<1:25:29, 60.35s/it]C:\Users\iamollas\anaconda3\lib\site-packages\joblib\externals\loky\process_e

In [12]:
results

[['1', '500', 'sqrt', 'NoRed', '_', '_', 0.33992896795272826, 0.0, 0.0, 0, 0],
 ['1',
  '500',
  'sqrt',
  'R1',
  'apriori',
  '_',
  0.3485003399848938,
  0.0,
  0.0,
  5.204415021800062,
  0.0],
 ['1',
  '500',
  'sqrt',
  'R1',
  'fpgrowth',
  '_',
  0.34799147605895997,
  0.0,
  0.0,
  5.204415021800062,
  0.0],
 ['1',
  '500',
  'sqrt',
  'R2',
  '_',
  '_',
  0.3708826494216919,
  4.28,
  263.84,
  5.204415021800062,
  1.7377631007855998],
 ['1',
  '500',
  'sqrt',
  'R3',
  '_',
  '_',
  0.37170494318008424,
  0.19,
  55.58,
  5.204415021800062,
  1.487352619274743],
 ['1', '500', 'log2', 'NoRed', '_', '_', 0.3452672171592712, 0.0, 0.0, 0, 0],
 ['1',
  '500',
  'log2',
  'R1',
  'apriori',
  '_',
  0.35465292930603026,
  0.0,
  0.0,
  5.204415021800063,
  0.0],
 ['1',
  '500',
  'log2',
  'R1',
  'fpgrowth',
  '_',
  0.3532030940055847,
  0.0,
  0.0,
  5.204415021800063,
  0.0],
 ['1',
  '500',
  'log2',
  'R2',
  '_',
  '_',
  0.38216103792190553,
  4.28,
  263.84,
  5.2044150